In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [212]:
import os
import librosa
import librosa.display
import numpy as np
train_audio_path = '../../Music_Genre_Classification/dataset/gtzan/'
feature_audio_path = '../../Music_Genre_Classification/dataset/feature/'
feature_name = 'train/hiphop/hiphop.00019.npy'
filename = 'hiphop/hiphop.00000.wav'
samples, sample_rate = librosa.load(str(train_audio_path)+filename)

In [213]:
genres = ['pop', 'metal', 'disco', 'reggae', 'classical', 'hiphop', 'country', 'jazz']

In [214]:
for genre in genres:
    temp = librosa.util.find_files(train_audio_path+genre)
    for ttemp in temp:
        samples, sample_rate = librosa.load(str(ttemp))
        print(genre,len(samples))
        break

pop 661504
metal 661504
disco 664180
reggae 661794
classical 661794
hiphop 661504
country 663300
jazz 661794


In [215]:
test = [1,2,3,4,5]
np.max(test)

5

In [216]:
npsample = np.load(os.path.join(feature_audio_path,feature_name))

In [217]:
temp = np.split(npsample,8)

In [224]:
from torch.nn.utils.rnn import PackedSequence, pack_padded_sequence, pad_packed_sequence

class CNN(nn.Module):
    def __init__(self):
        
        super(CNN,self).__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2))

        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2))
    
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=4))

        self.conv4 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=4)
        )
                
        self._classfier = nn.Sequential(
                nn.Linear(in_features=1024, out_features=512),
                nn.Linear(in_features=512, out_features=256),
                nn.Linear(in_features=256, out_features=8)
        )
    def forward(self,x):
        x = torch.unsqueeze(x,0)
        x =  torch.unsqueeze(x,0)
        print(x.shape)
        x = self.conv1(x)
        print(x.shape)
        x = self.conv2(x)
        print(x.shape)
        x = self.conv3(x)
        print(x.shape)
        x = self.conv4(x)
        print(x.shape)


        print(x.size(3),x.size(-1),x.permute(3, 0, 1, 2).shape)
        x = x.permute(3, 0, 1, 2).view(x.size(3), x.size(0), -1)
        print(x.shape)
        output, hn = self._rnnModule(x)
        x = self._classfier(output)
        
        return x

In [225]:
x = torch.tensor(temp[0]) 
x = torch.unsqueeze(x,0)
print(x.shape)
x = x.view(1,1,-1)
print(x.shape)

torch.Size([1, 128, 128])
torch.Size([1, 1, 16384])


In [226]:
net = CNN()
x = torch.tensor(temp[0]) 
net(x)

torch.Size([1, 1, 128, 128])
torch.Size([1, 64, 64, 64])
torch.Size([1, 128, 32, 32])
torch.Size([1, 256, 8, 8])
torch.Size([1, 512, 2, 2])
2 2 torch.Size([2, 1, 512, 2])
torch.Size([2, 1, 1024])


AttributeError: 'CNN' object has no attribute '_rnnModule'